<a href="https://colab.research.google.com/github/dp-93/Transfer-Learning-with-MobileNetV2/blob/main/Transfer_Learning_with_MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# --- The FINAL Training and Saving Script ---

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

# --- 1. Load the CIFAR-10 Dataset ---
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# --- 2. Resize the Images BEFORE Training ---
IMG_SIZE = 96
X_train_resized = tf.image.resize(X_train, (IMG_SIZE, IMG_SIZE))
X_test_resized = tf.image.resize(X_test, (IMG_SIZE, IMG_SIZE))

# --- 3. Load the Pre-trained MobileNetV2 Model ---
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

# --- 4. Build a Clean Model (No Lambda Layer) ---
# The model now directly expects the larger 96x96 images
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(10)
])

# --- 5. Compile and Train the Model ---
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print("\n--- Training the FINAL Clean Model ---")
model.fit(X_train_resized, y_train, epochs=10,
          validation_data=(X_test_resized, y_test))

# --- 6. Save the Clean Model ---
model.save('cifar10_transfer_learning_model_clean.keras')
print("\n--- Clean model saved successfully! ---")


--- Training the FINAL Clean Model ---
Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 37s 19ms/step - accuracy: 0.6431 - loss: 1.0562 - val_accuracy: 0.7890 - val_loss: 0.6153
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.7708 - loss: 0.6646 - val_accuracy: 0.7893 - val_loss: 0.6199
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.7813 - loss: 0.6369 - val_accuracy: 0.7886 - val_loss: 0.6105
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.7832 - loss: 0.6201 - val_accuracy: 0.7975 - val_loss: 0.5933
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.7918 - loss: 0.6026 - val_accuracy: 0.8030 - val_loss: 0.5808
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.7847 - loss: 0.6192 - val_accuracy: 0.7779 - val_loss: 0.6499
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.7883 - loss: 0.6116 - val_accuracy: 0.7967 - val_loss: 0.5874
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━

In [5]:
# --- The FINAL Deployment Script ---

import tensorflow as tf
from tensorflow import keras
import gradio as gr
import numpy as np
from PIL import Image

# --- 1. Load Your CLEAN Saved Model ---
print("Loading pre-trained model...")
loaded_model = keras.models.load_model('cifar10_transfer_learning_model_clean.keras')
print("Model loaded successfully!")


# --- 2. Define the Prediction Function (no changes needed here) ---
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
IMG_SIZE = 96

def predict_image(input_image):
    img_array = np.array(input_image)
    pil_image = Image.fromarray(img_array).resize((IMG_SIZE, IMG_SIZE))
    img_array = np.array(pil_image)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = loaded_model.predict(img_array)
    scores = tf.nn.softmax(predictions[0])
    confidence_scores = {class_names[i]: float(scores[i]) for i in range(10)}
    return confidence_scores


# --- 3. Install Gradio and Launch the App ---
!pip install -q gradio

interface = gr.Interface(fn=predict_image,
                         inputs=gr.Image(type="pil", label="Upload your image"),
                         outputs=gr.Label(num_top_classes=3, label="Top Predictions"),
                         title="CIFAR-10 Image Classifier (Transfer Learning)",
                         description="Upload an image to see the model's prediction.")

interface.launch()

Loading pre-trained model...
Model loaded successfully!
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff28b58962d6850cce.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
